# Example genespector session

In [1]:
import scanpy as sc
import genespector.app as gp

/Users/idk25/Dropbox/R2/genespector/genespector/app.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/Users/idk25/Dropbox/R2/genespector/genespector/app.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
adata = sc.read('./genespector/data/tiny_example2.h5ad')
adata

/usr/local/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/usr/local/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


AnnData object with n_obs × n_vars = 448 × 18247
    obs: 'isLK', 'gate', 'n_genes', 'n_counts', 'leiden', 'leiden_noG', 'random'
    var: 'symbol', 'geneid', 'isCC', 'n_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'draw_graph', 'leiden', 'leiden_colors', 'leiden_noG_colors', 'neighbors', 'pca'
    obsm: 'X_draw_graph_fa', 'X_draw_graph_fa_fromFIONA', 'X_draw_graph_fa_noG', 'X_umap', 'X_umap_noG'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

Generating + running is just one call:

In [3]:
gp.make_app(adata)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app 'genespector.app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)


Most AnnData files should work with genespector directly, in some cases preprocessing can be useful, see below.

## Preprocessing AnnData

### Using just sparse matrices (for large datasets)

When analysing large datasets note that using scaled values (in a dense matrix) is more memory-consuming. As an example we will generate an AnnData object with the scaled data in the .X slot and the log-normalised values in the .raw.X slot

In [3]:
adata.raw = adata
sc.pp.scale(adata)

To save resources we remove the scaled values and just use log-normalised values. It's a simple case of rebuilding the adata:


In [4]:
import anndata
adata2 = anndata.AnnData(X = adata.raw.X,
                   obs = adata.obs,
                   var = adata.raw.var,
                   uns = adata.uns,
                   obsm = adata.obsm)

In [5]:
adata2.X

<448x18247 sparse matrix of type '<class 'numpy.float32'>'
	with 1274989 stored elements in Compressed Sparse Row format>

### Categorical columns in the adata.obs

Only categorical columns from the .obs column are imported by genespector. To convert selected columns to categorical run:

In [6]:
adata.obs[['leiden', 'leiden_noG']] = adata.obs[['leiden', 'leiden_noG']].astype('category')

### Choosing coordinates

In some cases we may not want all the coordinates (e.g. 100 computed PCs) to be available from the dropdown menu. Here we will delete the X_umap_noG coordinates:

In [7]:
del adata.obsm['X_umap_noG']